# **Fact Orders**

### **Data Reading**

In [0]:
df = spark.sql("select * from project1_catalog.silver.orders_silver")
df.display()

In [0]:
df_dim_cus= spark.sql("select DimCustomersKey, customer_id as DimCustomerId from project1_catalog.gold.dimcustomers")

df_dim_prod= spark.sql("select product_id as DimProductId, product_id as DimProductKey from project1_catalog.gold.dimproducts")

In [0]:
df_fact = (
    df.join(df_dim_cus, df.customer_id == df_dim_cus.DimCustomerId, how="left")
      .join(df_dim_prod, df.product_id == df_dim_prod.DimProductId, how="left")
)

df_fact_new =  df_fact.drop('DimCustomerId', 'DimProductId', 'customer_id', 'product_id')

In [0]:
df_fact.printSchema()

In [0]:
df_fact_new.display()

In [0]:
df_fact_new.printSchema()

In [0]:
from delta.tables import DeltaTable

In [0]:
from delta.tables import DeltaTable

# Remove duplicates from the source DataFrame based on the keys used in the merge condition
df_fact_new_deduped = df_fact_new.dropDuplicates(["order_id", "DimCustomersKey", "DimProductKey"])

if spark.catalog.tableExists("project1_catalog.gold.FactOrders"):
    dlt_obj = DeltaTable.forName(spark, "project1_catalog.gold.FactOrders")

    dlt_obj.alias("trg").merge(
        df_fact_new_deduped.alias("src"),
        "trg.order_id=src.order_id AND trg.DimCustomersKey=src.DimCustomersKey AND trg.DimProductKey=src.DimProductKey"
    ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

else:
    df_fact_new_deduped.write.format("delta").option(
        "path", "abfss://gold@project1ds.dfs.core.windows.net/FactOrders"
    ).saveAsTable("project1_catalog.gold.FactOrders")

In [0]:
%sql
SELECT * FROM project1_catalog.GOLD.factorders;

In [0]:
# %sql drop table if exists project1_catalog.gold.FactOrders;